In [31]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, classification_report, roc_curve, auc
from catboost import CatBoostClassifier
import catboost
import optuna

Bootstrapping method to find the confidence interval of the AUROC score

In [32]:
def bootstrap_auc_ci(y_true, y_scores, n_bootstraps=2000, ci=0.95):
    rng = np.random.default_rng(42)
    aucs = []

    y_true = np.array(y_true)
    y_scores = np.array(y_scores)

    for _ in range(n_bootstraps):
        idx = rng.integers(0, len(y_true), len(y_true))
        if len(np.unique(y_true[idx])) < 2:
            continue
        aucs.append(roc_auc_score(y_true[idx], y_scores[idx]))

    lower = np.percentile(aucs, (1 - ci) / 2 * 100)
    upper = np.percentile(aucs, (1 + ci) / 2 * 100)
    return np.mean(aucs), lower, upper

Loading the dataset, pre-processing, and analysing the data

In [33]:
cohort_data = pd.read_csv('../cohort_data_new.csv')
cohort_data

,icustay_id,anion_gap_mean,anion_gap_sd,anion_gap_min,anion_gap_max,bicarbonate_mean,bicarbonate_sd,bicarbonate_min,bicarbonate_max,calcium_total_mean,...,urea_nitrogen_min,urea_nitrogen_max,white_blood_cells_mean,white_blood_cells_sd,white_blood_cells_min,white_blood_cells_max,age,gender,icu_los_hours,target
0,200003,13.375000,3.583195,9.0,21.0,25.250000,3.105295,18.0,28.0,7.771429,...,10.0,21.0,26.471429,13.176711,13.2,43.9,48,M,141,0
1,200007,15.500000,2.121320,14.0,17.0,23.000000,1.414214,22.0,24.0,8.900000,...,8.0,10.0,10.300000,1.272792,9.4,11.2,44,M,30,0
2,200009,9.500000,2.121320,8.0,11.0,23.333333,2.081666,21.0,25.0,8.000000,...,15.0,21.0,12.471429,1.471637,10.5,14.3,47,F,51,0
3,200012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,4.900000,NaN,4.9,4.9,33,F,10,0
4,200014,10.000000,1.732051,9.0,12.0,24.000000,1.000000,23.0,25.0,7.733333,...,21.0,24.0,13.233333,2.203028,10.7,14.7,85,M,41,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30484,299992,15.375000,2.856153,11.0,25.0,23.125000,2.609556,15.0,26.0,8.307143,...,8.0,23.0,14.134783,3.781727,8.1,22.1,41,M,499,0
30485,299993,9.400000,1.341641,8.0,11.0,29.600000,2.073644,26.0,31.0,8.000000,...,12.0,15.0,12.600000,0.605530,12.0,13.3,26,M,67,0
30486,299994,16.157895,2.477973,13.0,24.0,21.631579,3.451417,17.0,31.0,8.100000,...,28.0,63.0,10.076190,2.642329,5.3,14.5,74,F,152,1
30487,299998,11.500000,1.732051,10.0,14.0,23.500000,1.290994,22.0,25.0,8.800000,...,20.0,22.0,9.900000,1.210372,7.9,11.0,87,M,46,1


In [34]:
print(f"Dataset shape: {cohort_data.shape}")
print(f"Readmission rate: {cohort_data['target'].mean() * 100:.2f}%")

Dataset shape: (30489, 93)
Readmission rate: 10.74%


In [35]:
lab_cols = [
    'anion_gap_mean', 'anion_gap_min', 'anion_gap_max', 'anion_gap_sd',
    'bicarbonate_mean', 'bicarbonate_min', 'bicarbonate_max', 'bicarbonate_sd',
    'calcium_total_mean', 'calcium_total_min', 'calcium_total_max', 'calcium_total_sd',
    'chloride_mean', 'chloride_min', 'chloride_max', 'chloride_sd',
    'creatinine_mean', 'creatinine_min', 'creatinine_max', 'creatinine_sd',
    'glucose_mean', 'glucose_min', 'glucose_max', 'glucose_sd',
    'hematocrit_mean', 'hematocrit_min', 'hematocrit_max', 'hematocrit_sd',
    'hemoglobin_mean', 'hemoglobin_min', 'hemoglobin_max', 'hemoglobin_sd',
    'mchc_mean', 'mchc_min', 'mchc_max', 'mchc_sd',
    'mcv_mean', 'mcv_min', 'mcv_max', 'mcv_sd',
    'magnesium_mean', 'magnesium_min', 'magnesium_max', 'magnesium_sd',
    'pt_mean', 'pt_min', 'pt_max', 'pt_sd',
    'phosphate_mean', 'phosphate_min', 'phosphate_max', 'phosphate_sd',
    'platelet_count_mean', 'platelet_count_min', 'platelet_count_max', 'platelet_count_sd',
    'potassium_mean', 'potassium_min', 'potassium_max', 'potassium_sd',
    'rdw_mean', 'rdw_min', 'rdw_max', 'rdw_sd',
    'red_blood_cells_mean', 'red_blood_cells_min', 'red_blood_cells_max', 'red_blood_cells_sd',
    'sodium_mean', 'sodium_min', 'sodium_max', 'sodium_sd',
    'urea_nitrogen_mean', 'urea_nitrogen_min', 'urea_nitrogen_max', 'urea_nitrogen_sd',
    'white_blood_cells_mean', 'white_blood_cells_min', 'white_blood_cells_max', 'white_blood_cells_sd',
    'age', 'icu_los_hours'
]

REmove the ICUstay_id and the gender

In [36]:
drop_cols = [c for c in cohort_data.columns if 'icustay_id' in c.lower() or 'gender' in c.lower()]
df = cohort_data.drop(columns=['icustay_id', 'gender'], errors='ignore')

X = df.drop(columns=['target'])
y = df['target']
X

,anion_gap_mean,anion_gap_sd,anion_gap_min,anion_gap_max,bicarbonate_mean,bicarbonate_sd,bicarbonate_min,bicarbonate_max,calcium_total_mean,calcium_total_sd,...,urea_nitrogen_mean,urea_nitrogen_sd,urea_nitrogen_min,urea_nitrogen_max,white_blood_cells_mean,white_blood_cells_sd,white_blood_cells_min,white_blood_cells_max,age,icu_los_hours
0,13.375000,3.583195,9.0,21.0,25.250000,3.105295,18.0,28.0,7.771429,0.292770,...,15.571429,4.577377,10.0,21.0,26.471429,13.176711,13.2,43.9,48,141
1,15.500000,2.121320,14.0,17.0,23.000000,1.414214,22.0,24.0,8.900000,NaN,...,9.000000,1.414214,8.0,10.0,10.300000,1.272792,9.4,11.2,44,30
2,9.500000,2.121320,8.0,11.0,23.333333,2.081666,21.0,25.0,8.000000,NaN,...,17.333333,3.214550,15.0,21.0,12.471429,1.471637,10.5,14.3,47,51
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,4.900000,NaN,4.9,4.9,33,10
4,10.000000,1.732051,9.0,12.0,24.000000,1.000000,23.0,25.0,7.733333,0.057735,...,23.000000,1.732051,21.0,24.0,13.233333,2.203028,10.7,14.7,85,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30484,15.375000,2.856153,11.0,25.0,23.125000,2.609556,15.0,26.0,8.307143,0.255597,...,16.000000,4.662524,8.0,23.0,14.134783,3.781727,8.1,22.1,41,499
30485,9.400000,1.341641,8.0,11.0,29.600000,2.073644,26.0,31.0,8.000000,0.216025,...,13.000000,1.224745,12.0,15.0,12.600000,0.605530,12.0,13.3,26,67
30486,16.157895,2.477973,13.0,24.0,21.631579,3.451417,17.0,31.0,8.100000,0.316228,...,44.578947,12.102873,28.0,63.0,10.076190,2.642329,5.3,14.5,74,152
30487,11.500000,1.732051,10.0,14.0,23.500000,1.290994,22.0,25.0,8.800000,0.416333,...,20.750000,0.957427,20.0,22.0,9.900000,1.210372,7.9,11.0,87,46


trying out feature engineering

In [37]:
# X["renal_ratio"] = X["urea_nitrogen_mean"] / (X["creatinine_mean"] + 1e-5)
# X["creatinine_range"] = X["creatinine_max"] - X["creatinine_min"]
# X["bun_range"] = X["urea_nitrogen_max"] - X["urea_nitrogen_min"]

# X["anion_gap_bicarb_ratio"] = X["anion_gap_mean"] / (X["bicarbonate_mean"] + 1e-5)
# X["anion_gap_range"] = X["anion_gap_max"] - X["anion_gap_min"]

# X["hb_hct_ratio"] = X["hemoglobin_mean"] / (X["hematocrit_mean"] + 1e-5)
# X["rbc_variability"] = X["red_blood_cells_sd"]
# X["plt_range"] = X["platelet_count_max"] - X["platelet_count_min"]

# X["sodium_range"] = X["sodium_max"] - X["sodium_min"]
# X["potassium_range"] = X["potassium_max"] - X["potassium_min"]
# X["chloride_range"] = X["chloride_max"] - X["chloride_min"]

# X["glucose_range"] = X["glucose_max"] - X["glucose_min"]
# X["glucose_instability"] = X["glucose_sd"]

# X["age_los_interaction"] = X["age"] * X["icu_los_hours"]
# X["icu_los_log"] = np.log1p(X["icu_los_hours"])

# X

In [38]:
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.30, random_state=7, stratify=y, shuffle=True)
X_train, X_eval, y_train, y_eval = train_test_split(X_train_full, y_train_full, test_size=0.30, random_state=7, stratify=y_train_full, shuffle=True)

In [39]:
def objective(trial):

    params = {
        "iterations": trial.suggest_int("iterations", 500, 3000),
        "learning_rate": trial.suggest_float("learning_rate", 1e-3, 0.3, log=True),
        "depth": trial.suggest_int("depth", 4, 10),
        "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-2, 10, log=True),
        "bagging_temperature": trial.suggest_float("bagging_temperature", 0, 5),
        "random_strength": trial.suggest_float("random_strength", 0, 2),
        "border_count": trial.suggest_int("border_count", 32, 255),
        "random_seed": 42,
        "loss_function": "Logloss",
        "eval_metric": "AUC",
        "task_type": "CPU",
        "verbose": False,
        "od_type": "Iter",
        "od_wait": 120
    }

    model = CatBoostClassifier(**params)
    model.fit(catboost.Pool(X_train, y_train), eval_set=catboost.Pool(X_eval, y_eval), verbose=False)
    pred_val = model.predict_proba(X_eval)[:, 1]
    val_auc = roc_auc_score(y_eval, pred_val)

    return val_auc

hyperparameter search - TPE

In [40]:
sampler = optuna.samplers.TPESampler(seed=7)
study = optuna.create_study(direction="maximize", sampler=sampler)

study.optimize(objective, n_trials=50, show_progress_bar=True)

print("Best Trial:")
print(study.best_trial.params)

[I 2025-12-01 13:08:07,352] A new study created in memory with name: no-name-49af8425-64a7-4862-b629-96450d32c2c3


  0%|          | 0/50 [00:00<?, ?it/s]

[I 2025-12-01 13:08:11,750] Trial 0 finished with value: 0.6965660542432196 and parameters: {'iterations': 690, 'learning_rate': 0.08549750026376444, 'depth': 7, 'l2_leaf_reg': 1.4804560990495679, 'bagging_temperature': 4.889947559983013, 'random_strength': 1.0769917408208673, 'border_count': 144}. Best is trial 0 with value: 0.6965660542432196.
[I 2025-12-01 13:08:20,366] Trial 1 finished with value: 0.6974851980711713 and parameters: {'iterations': 680, 'learning_rate': 0.004623340751759673, 'depth': 7, 'l2_leaf_reg': 1.090661513591819, 'bagging_temperature': 4.018695180521878, 'random_strength': 0.7618822662970768, 'border_count': 46}. Best is trial 1 with value: 0.6974851980711713.
[I 2025-12-01 13:08:23,193] Trial 2 finished with value: 0.6929151661274899 and parameters: {'iterations': 1220, 'learning_rate': 0.17913211790965036, 'depth': 5, 'l2_leaf_reg': 0.22718093648132454, 'bagging_temperature': 4.656030098445108, 'random_strength': 0.049798455100696026, 'border_count': 166}. B

Retraining using the best parameters

In [41]:
best_params = study.best_trial.params
best_params.update({
    "loss_function": "Logloss",
    "eval_metric": "AUC",
    "random_seed": 7,
    "task_type": "CPU",
    "verbose": 200
})

final_model = CatBoostClassifier(**best_params)
final_model.fit(X_train_full, y_train_full)

0:	total: 13.8ms	remaining: 34.6s
200:	total: 2.8s	remaining: 32.1s
400:	total: 5.58s	remaining: 29.3s
600:	total: 8.39s	remaining: 26.6s
800:	total: 11.1s	remaining: 23.6s
1000:	total: 13.8s	remaining: 20.7s
1200:	total: 16.6s	remaining: 18s
1400:	total: 19.2s	remaining: 15.2s
1600:	total: 21.9s	remaining: 12.4s
1800:	total: 24.6s	remaining: 9.63s
2000:	total: 27.4s	remaining: 6.91s
2200:	total: 30.1s	remaining: 4.16s
2400:	total: 32.8s	remaining: 1.42s
2504:	total: 34.3s	remaining: 0us


Evaluate

In [42]:
# calculate training error
y_train_pred = final_model.predict(X_train_full)
train_error = np.mean(y_train_pred != y_train_full)
print(f"Training error (Vanilla LR): {train_error:.3f}")

# calculate test error
y_pred = final_model.predict(X_test)
y_pred_proba = final_model.predict_proba(X_test)[:, 1]
test_error = np.mean(y_pred != y_test)
print(f"Test error (Vanilla LR): {test_error:.3f}")

Training error (Vanilla LR): 0.095
Test error (Vanilla LR): 0.107


In [43]:
print("Classification Report:\n")
print(classification_report(y_test, y_pred, target_names=["No Readmission (0)", "Readmission (1)"]))

fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba, pos_label=1)
roc_auc = auc(fpr, tpr)
np.savez('../results/catboost_hyperOpt_fpr_tpr_thresholds.npz', fpr, tpr, thresholds)
print(f"AUROC for class 1 (Readmission): {roc_auc:.3f}")

auc, lower, upper = bootstrap_auc_ci(y_test, y_pred_proba)
print(f"AUC = {auc:.4f}, 95% CI = [{lower:.4f}, {upper:.4f}]")

Classification Report:

                    precision    recall  f1-score   support

No Readmission (0)       0.89      1.00      0.94      8164
   Readmission (1)       0.62      0.01      0.03       983

          accuracy                           0.89      9147
         macro avg       0.76      0.51      0.48      9147
      weighted avg       0.86      0.89      0.84      9147

AUROC for class 1 (Readmission): 0.737
AUC = 0.7364, 95% CI = [0.7204, 0.7528]
